## Drugs Arrested Case

### ปัจจัยที่ส่งผลต่อจำนวนคดีผู้ต้องหายาเสพติด
#### by กลุ่ม ทุกปัญหามีทางเข้าเสมอ
---

In [ ]:
import pandas as pd
import numpy as np
import requests
import json, io, time, urllib3, re
import matplotlib.pyplot as plt
import seaborn as sns
from ipywidgets import widgets, interact
urllib3.disable_warnings()

เกริ่นเรื่องปัญหายาเสพติดในไทย

- จำนวนคดีเคส เรียงตามจังหวัด เอาปีล่าสุด - เลือกจังหวัดแสดงผลได้
- ดูเทรนด์ rank top 3 แต่ละปี
- เอาจังหวัดที่น่าสนใจ(สถิติอยู่ในท็อป3บ่อย) ex.นครศรีธรรมราช กรุงเทพ ปทุม สงขลา ขอนแก่น
- ประชากรหนาแน่น ex.กรุงเทพ ขอนแก่น
- ดูเรื่องนักท่องเที่ยว ฟีลเมืองเศรษฐกิจ ex.ชลบุรี
- หัวข้อภาคใต้ เน้นนครศรี + เรื่องกระท่อม + อาชญากรรมอื่น
- การกินเหล้า & เล่นยาสวนทางกัน

### Overview

ปัญหายาเสพติดเป็นเหมือนโรคเรื้อรังที่กัดกินอนาคตของชาติไทยมานานหลายทศวรรษ แม้จะมีความพยายามจากทุกภาคส่วนในการแก้ไข แต่ปัญหานี้ยังคงฝังรากลึกในสังคมไทย

วันนี้กลุ่มของพวกเราจะมานำเสนอข้อมูลเกี่ยวกับยาเสพติด โดยเน้นไปที่จำนวนคดียาเสพติดในแต่ละจังหวัด พร้อมกับค้นหาว่าปัจจัยใดบ้างที่ส่งผลต่อจำนวนความมากน้อยของปัญหายาเสพติดได้ รวมไปถึงหยิบยก insights ที่น่าสนใจ ซึ่งเป็นสาเหตุที่ซ่อนอยู่ในตัวเลขยอดคดียาเสพติดของบางจังหวัดมานำเสนอเพิ่มเติม

*หมายเหตุ ข้อมูลจำนวนผู้ต้องหาคดียาเสพติดเป็นเพียงตัวเลขหนึ่งที่สะท้อนถึงปัญหายาเสพติด ว่ามีมากน้อยเพียงใด มิใช่ตัวเลขที่เป็นตัวแทนสะท้อนข้อมูลออกมาได้ทั้งหมด 


### จำนวนคดีผู้ต้องหาคดียาเสพติด เรียงตามจังหวัด ปี 2565

In [ ]:
df = pd.read_csv("./data/arrested_drug_cases.csv", encoding = "utf8")
df["budgetYear"] = df["budgetYear"].astype(str)

df_prov_th_en = pd.read_csv("./data/province_th_en.csv", encoding = "utf8")
df = pd.merge(df, df_prov_th_en, on='PROV_NAME', how='left').copy()
df.head()

- bar chart แสดงเปรียบเทียบ 10 จังหวัดที่มีจำนวนคดียาเสพติดสูงสุด ในปี 2565

In [ ]:
df_for_bar = df[['budgetYear', 'PROV_NAME', 'arrestAll_case', 'arrestAll_per', 'prov_en']][df["budgetYear"] == "2565"].copy()
df_for_bar.sort_values("arrestAll_case", ascending = False, inplace = True)

df_top_x = df_for_bar[:10]

sns.barplot(
    data = df_top_x,
    x = "arrestAll_case",
    y = "prov_en",
    palette = ["#a91616" for _ in range(len(df_top_x))], # just to eliminate warning
    hue = "prov_en",
)

plt.xlabel("Cases")
plt.ylabel("Province")
plt.title("Top 10 Highest Drug Arrested Case by Province in 2565")
plt.show()

- แยกดูรายจังหวัดทุกปีได้

In [6]:
df_interact_prov = df[['budgetYear', 'PROV_NAME', 'arrestAll_case', 'arrestAll_per', 'prov_en']].copy()
provinces = sorted(df_interact_prov["PROV_NAME"].unique().tolist())

@interact
def bar_by_province(province = provinces):
    data = df_interact_prov[df["PROV_NAME"] == province]

    # average 2 years prior and after to fill the missing value
    avg = data[data["budgetYear"].isin(["2557", "2558", "2560", "2561"])]["arrestAll_case"].mean()
    data.loc[data["arrestAll_case"] == 0, "arrestAll_case"] = avg.astype(int)

    g = sns.barplot(
        data = data,
        x = "budgetYear",
        y = "arrestAll_case",
        palette = ["#a91616" for _ in range(len(data))], # just to eliminate warning
        hue = "budgetYear",
    )

    prov_en = data["prov_en"].iloc[0]

    plt.ylabel("Cases")
    plt.xlabel("Year")
    plt.title(f"Number of Cases over the time in {prov_en}")

#### Top 3 Province with Highest Drugs Arrested Case 

In [ ]:
Top3 = df[["budgetYear", "PROV_NAME", "arrestAll_case"]].copy()
Top3["arrestAll_case"] = Top3["arrestAll_case"].astype(int)
Top3['rank'] = Top3.groupby(["budgetYear"])['arrestAll_case'].rank(method='min', ascending = False).copy()
RankTop3 = Top3[["budgetYear", "PROV_NAME", "arrestAll_case", "rank"]].copy()
RankTop3[RankTop3["rank"] <= 3].sort_values(["budgetYear", "rank"])

In [ ]:
RankTop3[RankTop3["rank"] <= 3].sort_values(["budgetYear", "rank"])
@interact 
def filter_year(year=RankTop3["budgetYear"].unique()): 
    return RankTop3[RankTop3["budgetYear"]== year].sort_values(["budgetYear", "rank"]) 

#### ปัจจัยความหนาแน่นประชากร

#### ปัจจัยทางเศรษฐกิจ การท่องเที่ยว
- กรุงเทพ เชียงใหม่ สงขลา ขอนแก่น  (เดี๋ยวลองดูว่ามีตัวเลขเพิ่มเติมมั้ยมาให้เห็นว่าเป็นจังหวัดท่องเที่ยว) + ตรงนี้อาจจะเอากราฟมาให้ดูอีกที??
    ถ้าสังเกตุข้อมูลข้าง้ตนที่ได้นำเสนอไป จะเห็นได้ว่ามีหลายจังหวัด ใน Top 10 Highest  Drug Arrested Case นั้นเป็นประเทศที่มีความโดดเด่น และเป็นจังหวัดแห่งการท่องเที่ยวทั้งสำหรับคนไทย และต่างชาติ

    ถึงแม้ว่าจะมีจังหวัดภูเก็ต

#### Southern Province Insights
พวกเราอาจเคยได้ยินข่าวเรื่องการลักลอบนำเข้าขนส่งยาเสพติดกันบ่อยๆ ตามแถบชายแดนติดฝั่งพม่า หรือทางฝั่งภาคเหนือ

แต่จากข้อมูลจะพบได้ว่าจังหวัดในภาคใต้ก็มีจำนวนปัญหายาเสพติดไม่น้อยหน้าเช่นกัน

เวลาจับกุมคดีเกี่ยวกับยาเสพติดได้นั้น ไม่ว่าจะจับได้ที่ส่วนไหนของประเทศไทยผู้ต้องหามักจะให้การว่าจะส่งยาไปยังชายแดนใต้ ถึงแม้ว่าตรงชายแดนใต้จะไม่ได้มีการตั้งโรงงานผลิตยา แต่เป็นพื้นที่พักยา เพื่อทำการส่งไปยังประเทศมาเลยเซีย และทวีปอื่นๆทั่วโลก

โดนหนึ่งในสาเหตุที่ทำให้ยาเสพติดแพร่ระบาดเยอะคือกระบวนการค้ายาเสพติดถูกสกัดจับได้เพียงแค่ประมาณ 10% จากปริมารการลักลอบนำเข้าทั้งหมด
และนอกการการเสพติดพวกยาบ้า ยาไอซ์ ฝิ่นแล้ว ยังเป็นพื้นที่ปลูกพืชยาเสพติด โดยพืชกระท่อมนับเป็นสิ่งที่นิยมกันมากในภาคใต้ เนื่องจากมีราคาถูก

-   จังหวัด สงขลา 

    ในจังหวัดสงขลามีช่องทางสำหรับการลักลอบนำเข้ายาเสพติดจำนวนมาก ทั้งด่านที่เป็นด่านศุลการกร ด่านตรวจคนเข้าเมือง จุดตรวจพรมแดนต่างๆ รวมถึงมีช่องทางธรรมชาติ และยังเป็นจังหวัดที่มีชายแดนติดกับประเทศมาเลเซียด้วย สามารถเข้าออกประเทศกันได้ง่าย 


#### Reversion of Drugs & Alcohol Consumption (ตรงนี้มีตัวเลขอะไรมาสนับสนุนได้มั้ยอ่ะ)

ยาเสพติดและเครื่องดื่มแอลกอฮอล์ต่างก็เป็นสารเสพติดเช่นเดียวกัน ดังนั้นหลายคนจึงอาจจะคิดว่าปัญหายาเสพติดและปัญหาการบริโภคเครื่องดื่มแอลกอฮอล์นั้นควรจะมีแนวโน้มไปในทิศทางเดียวกัน ผู้ที่ติดยาเสพติดส่วนใหญ่ก็มีแนวโน้มที่จะติดสุราและดื่มเครื่องดื่มแอลกอฮอล์เป็นประจำด้วย 

รวมไปถึงผลสำรวจก็พบว่า การติดยาเสพติดโดยเฉพาะในกลุ่มวัยรุ่นจะเริ่มจากการดื่มเครื่องดื่มแอลกอฮอล์ การสูบบุหรี่ ก่อนจะนำไปสู่สารเสพติดชนิดอื่นๆที่มีความรุนแรงขึ้นเรื่อยๆ

แต่จากข้อมูลกลับมีจุดน่าสังเกตที่ว่า ภูมิภาคใต้ที่มีผู้ดื่มสุราหรือเครื่องดื่มแอลกอฮอล์และมีความเสี่ยงต่อปัญหาแอลกอฮอล์น้อยที่สุดกลับเป็นภูมิภาคที่มีคดียาเสพติดสูงที่สุด 
เช่น จังหวัดนครศรีธรรมราช สงขลา สุราษฎร์ธานี ซึ่งเป็นจังหวัดที่มีคดียาเสพติดสูงสุด 

ในทางกลับกัน จังหวัดที่อยู่ในภาคเหนือ เช่น แม่ฮ่องสอน แพร่ น่าน เชียงราย ที่มีปัญหายาเสพติดน้อย กลับเป็นจังหวัดที่มีความเสี่ยงต่อปัญหาแอลกอฮอล์สูงสุดในประเทศไทย

### ข้อเสนอแนะ

### References
- https://alcoholstudy.in.th/index.php?ct=report_drinker_province&proc=main&year=2564&chart_id=5&len=10&page=0&key=
- https://www.thaihealth.or.th/%E0%B8%AB%E0%B9%88%E0%B8%A7%E0%B8%87%E0%B8%9B%E0%B8%B1%E0%B8%8D%E0%B8%AB%E0%B8%B2%E0%B8%A2%E0%B8%B2%E0%B9%80%E0%B8%AA%E0%B8%9E%E0%B8%95%E0%B8%B4%E0%B8%94-%E0%B8%A3%E0%B8%B0%E0%B8%9A%E0%B8%B2%E0%B8%94/
- https://www.phufaresthome.com/blog/drug-issue-in-thai-teenager/
- https://www.agenda.co.th/social/alcohol-consumption-thailand/